# Introduction of Natural Language Processing(NLP)

In [65]:
import pandas as pd
import math
import re
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *
from sklearn.datasets import load_files

## [Task 1] Bag-of-Words(BoW) and N-gram

<br />

I am going to remove special characters such as ! and 〜, and split words, then quantify/vectorize the following corpus by BoW(1-gram) and BoW(2-gram).

1: 今撮影中で〜す！


2: 今休憩中で〜す(^^)


3: 今日ドラマ撮影で〜す！


4: 今日、映画瞬公開で〜す！！！

#### BoW(1-gram)

In [2]:
# Example

vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data=texts_vec, columns=vocabulary, index=ms_kk_texts)

In [3]:
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


In [4]:
# Above four sentences

voc_1g = ["今", "撮影", "中", "です", "休憩", "今日", "ドラマ", "映画", "瞬", "公開"]
texts_1g = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]
vec_1g = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,1,0,1,0,1,1,0,0,0], [0,0,0,1,0,1,0,1,1,1]]

df_bow_1g = pd.DataFrame(data=vec_1g, columns=voc_1g, index=texts_1g)

In [5]:
df_bow_1g

,今,撮影,中,です,休憩,今日,ドラマ,映画,瞬,公開
今撮影中で〜す！,1,1,1,1,0,0,0,0,0,0
今休憩中で〜す(^^),1,0,1,1,1,0,0,0,0,0
今日ドラマ撮影で〜す！,0,1,0,1,0,1,1,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,1,0,1,0,1,1,1


#### BoW(2-gram)

In [6]:
# Above four sentences

voc_2g = ["今撮影", "撮影中", "中です", "今休憩", "休憩中", "今日ドラマ", "ドラマ撮影", "撮影です", "今日映画", "映画瞬", "瞬公開", "公開です"]
texts_2g = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]
vec_2g = [[1,1,1,0,0,0,0,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0,0,0], [0,0,0,0,0,1,1,1,0,0,0,0], [0,0,0,0,0,0,0,0,1,1,1,1]]

df_bow_2g = pd.DataFrame(data=vec_2g, columns=voc_2g, index=texts_2g)

In [7]:
df_bow_2g

,今撮影,撮影中,中です,今休憩,休憩中,今日ドラマ,ドラマ撮影,撮影です,今日映画,映画瞬,瞬公開,公開です
今撮影中で〜す！,1,1,1,0,0,0,0,0,0,0,0,0
今休憩中で〜す(^^),0,0,1,1,1,0,0,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,1,1,1,0,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,0,0,0,0,0,1,1,1,1


## [Task 2] TF-IDF

<br />

I will quantify/vectorize the corpus of the 1st task by TF-IDF.

#### Standard Formula of TF-IDF

Term Frequency(TF):

$$
tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}
$$

$n_{t,d}$ : Number of words "t" on a document "d"

$\sum_{s \in d}n_{s,d}$ : Sum of number of all words on a document "d"

Inverse Document Frequency(IDF):

$$
idf(t) = \log{\frac{N}{df(t)}}
$$

$N$ : Number of all documents

$df(t)$ : Number of documents that a word "t" appears

TF-IDF:

$$
tfidf(t, d) = tf(t, d) \times idf(t)
$$

In [8]:
# Above four sentences

original_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]

voc_tf_idf = ["今", "撮影", "中", "です", "休憩", "今日", "ドラマ", "映画", "瞬", "公開"]
texts_tf_idf = ["今撮影中です", "今休憩中です", "今日ドラマ撮影です", "今日映画瞬公開です"]
vec_tf_idf = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,1,0,1,0,1,1,0,0,0], [0,0,0,1,0,1,0,1,1,1]]

In [9]:
# Compute TF

tf_list = []
for i in range(len(texts_tf_idf)):
    tf = []
    for j in range(len(voc_tf_idf)):
        tf.append(texts_tf_idf[i].count(voc_tf_idf[j]) / vec_tf_idf[i].count(1))
    tf_list.append(tf)

In [10]:
tf_list

[[0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.0, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.25, 0.0, 0.25, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0],
 [0.2, 0.0, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2]]

In [11]:
# Compute IDF

idf_list = []
for j in range(len(vec_tf_idf[0])):
    count = 0
    for i in range(len(texts_tf_idf)):
        if vec_tf_idf[i][j] == 1:
            count += 1
    idf = math.log2(len(texts_tf_idf) / count)
    idf_list.append(idf)

In [12]:
idf_list

[1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0]

In [13]:
# Compute TF-IDF

tf_idf_list = []
for i in range(len(tf_list)):
    tf_idf = []
    for j in range(len(idf_list)):
        tf_idf.append(tf_list[i][j] * idf_list[j])
    tf_idf_list.append(tf_idf)

In [14]:
tf_idf_list

[[0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.0, 0.25, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.25, 0.0, 0.0, 0.0, 0.25, 0.5, 0.0, 0.0, 0.0],
 [0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.4, 0.4, 0.4]]

In [15]:
# Dataframe

df_tf_idf = pd.DataFrame(data=tf_idf_list, columns=voc_tf_idf, index=original_texts)

In [16]:
df_tf_idf

,今,撮影,中,です,休憩,今日,ドラマ,映画,瞬,公開
今撮影中で〜す！,0.25,0.25,0.25,0.0,0.0,0.00,0.0,0.0,0.0,0.0
今休憩中で〜す(^^),0.25,0.00,0.25,0.0,0.5,0.00,0.0,0.0,0.0,0.0
今日ドラマ撮影で〜す！,0.25,0.25,0.00,0.0,0.0,0.25,0.5,0.0,0.0,0.0
今日、映画瞬公開で〜す！！！,0.20,0.00,0.00,0.0,0.0,0.20,0.0,0.4,0.4,0.4


## [Task 3] Text Cleaning

<br />

I am going to remove urls, 【◯◯】, special characters including line breaks and emojis.

In [144]:
# Target
text = "<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"

# <◯◯>
text = re.sub(r"<(.+)>", "", text)
# 【◯◯】
text = re.sub(r"【(.+)】", "", text)
# Urls
text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+", "", text)
# Emojis
text = re.sub(r":([a-xA-Z0-9_]+):", "", text)
# Special characters
text = re.sub(r"[\n\*`\s]", "", text)

In [145]:
text

'有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！'

In [146]:
# Check

correct_text = "有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！"

text == correct_text

True

## [Task 4] Morphological Analysis

<br />

There are tools for morphological analysis, like Mecab and Janome. In this task, I will use Janome.


https://mocobeta.github.io/janome/

<br />

I will do morphological analysis about the text that I did text cleaning on the previous task to extract nouns and verbs by Janome.

In [60]:
t = Tokenizer()
for token in t.tokenize(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ご	接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
！	記号,一般,*,*,*,*,！,！,！
この	連体詞,*,*,*,*,*,この,コノ,コノ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
、	記号,読点,*,*,*,*,、,、,、
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
で	助

In [59]:
# Nouns

token_filters = [POSKeepFilter("名詞")]
a = Analyzer(token_filters=token_filters)
for token in a.analyze(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
JetBrains	名詞,一般,*,*,*,*,JetBrains,*,*
社	名詞,接尾,一般,*,*,*,社,シャ,シャ
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
コード	名詞,一般,*,*,*,*,コード,コード,コード
提供	名詞,サ変接続,*,*,*,*,提供,テイキョウ,テイキョー
交渉	名詞,サ変接続,*,*,*,*,交渉,コウショウ,コーショー
100	名詞,数,*,*,*,*,100,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
こと	名詞,非自立,一般,*,*,*,こと,コト,コト
RubyMinePyCharm	名詞,一般,*,*,*,*,RubyMinePyCharm,*,*
希望	名詞,サ変接続,*,*,*,*,希望,キ

In [61]:
# Verbs

token_filters = [POSKeepFilter("動詞")]
a = Analyzer(token_filters=token_filters)
for token in a.analyze(text):
    print(token)

いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
実り	動詞,自立,*,*,五段・ラ行,連用形,実る,ミノリ,ミノリ
いただく	動詞,自立,*,*,五段・カ行イ音便,基本形,いただく,イタダク,イタダク
でき	動詞,自立,*,*,一段,連用形,できる,デキ,デキ
挙げ	動詞,自立,*,*,一段,連用形,挙げる,アゲ,アゲ
あげ	動詞,自立,*,*,一段,連用形,あげる,アゲ,アゲ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ください	動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,くださる,クダサイ,クダサイ
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
せ	動詞,接尾,*,*,一段,連用形,せる,セ,セ
いただき	動詞,非自立,*,*,五段・カ行イ音便,連用形,いただく,イタダキ,イタダキ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
おり	動詞,非自立,*,*,五段・ラ行,連用形,おる,オリ,オリ


## [Task 5] Analyze News

<br />

I will analyze some news of "livedoor" from various viewpoints.


http://news.livedoor.com/

In [64]:
# Show an explanation of the "livedoor" news

cat text/README.txt

livedoor ニュースコーパス


概要
-----------------
本コーパスは、NHN Japan株式会社が運営する「livedoor ニュース」のうち、下記のクリエイティブ・コモンズ
ライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。

- トピックニュース
  http://news.livedoor.com/category/vender/news/
- Sports Watch
  http://news.livedoor.com/category/vender/208/
- ITライフハック
  http://news.livedoor.com/category/vender/223/
- 家電チャンネル
  http://news.livedoor.com/category/vender/kadench/
- MOVIE ENTER
  http://news.livedoor.com/category/vender/movie_enter/
- 独女通信
  http://news.livedoor.com/category/vender/90/
- エスマックス
  http://news.livedoor.com/category/vender/smax/
- livedoor HOMME
  http://news.livedoor.com/category/vender/homme/
- Peachy
  http://news.livedoor.com/category/vender/ldgirls/

収集時期：2012年9月上旬


ライセンス
-----------------
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 - 改変禁止」
（http://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。
クレジット表示についてはニュースカテゴリにより異なるため、サブディレクトリにある
それぞれの LICENSE.txt をご覧ください。

livedoor はNHN Japan株式会社の登録商標です。


記事ファイルのフォーマット
-----------------
記事ファイルは以下のフォーマットにしたが

In [66]:
# Read files
bin_data = load_files('./text', encoding='utf-8')

documents = bin_data.data

# Get labels while will not use them this time.
targets = bin_data.target

In [72]:
# Check

print(documents[0])

http://news.livedoor.com/article/detail/4931238/
2010-08-08T10:00:00+0900
NY名物イベントが日本でも！名店グルメを気軽に楽しむ
ニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。
　
　実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。

参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。
　イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。

　予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？

JAPAN RESTAURANT WEEK 2010 -公式サイト



In [99]:
documents[0]

'http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n'

In [73]:
targets

array([5, 8, 7, ..., 1, 3, 3])

In [83]:
# Analyze those news by counting vocabularies

token_filters1 = [TokenCountFilter()]
a1 = Analyzer(token_filters=token_filters1)
dic = {}
for doc in documents:
    for k, v in a1.analyze(doc):
        if k not in dic:
            dic[k] = v
        else:
            dic[k] += v

In [84]:
print(dic)

{'http': 8340, '://': 8374, 'news': 7521, '.': 27963, 'livedoor': 8605, 'com': 8064, '/': 35330, 'article': 7462, 'detail': 7509, '4931238': 1, '\n': 75755, '2010': 1729, '-': 30810, '08': 1676, 'T': 7813, '10': 5335, ':': 16201, '00': 11246, '+': 7653, '0900': 7368, 'NY': 53, '名物': 33, 'イベント': 1423, 'が': 116334, '日本': 5379, 'で': 84555, 'も': 47548, '！': 19974, '名店': 11, 'グルメ': 82, 'を': 126780, '気軽': 255, 'に': 131630, '楽しむ': 539, 'ニューヨーク': 180, '20': 2434, '年': 9012, '続い': 366, 'て': 101450, 'いる': 26871, '食': 273, 'の': 207058, '祭典': 58, '「': 63155, 'レストラン': 187, '・': 41657, 'ウィーク': 21, '」': 62506, '。': 136136, 'その': 7138, '版': 1833, 'ダイナーズクラブ': 1, '特別': 713, '協賛': 30, 'もと': 250, '7': 3513, '月': 9498, '30': 3326, '日': 13651, 'より': 4682, 'スタート': 639, '8': 3308, '31': 727, 'まで': 6304, '期間': 936, '中': 6858, '、': 206233, '青山': 82, '六本木': 174, '丸の内': 103, '銀座': 292, '横浜': 216, 'エリア': 441, 'から': 21775, 'ラグジュアリーレストラン': 1, '81': 23, '店舗': 554, 'この': 7583, 'ため': 4945, '用意': 632, 'し': 62485, 'た': 8

In [97]:
# Sort

dic_sorted = sorted(dic.items(), key=lambda x: x[1], reverse=True)

In [98]:
dic_sorted

[('の', 207058),
 ('、', 206233),
 ('。', 136136),
 ('に', 131630),
 ('を', 126780),
 ('が', 116334),
 ('は', 112058),
 ('て', 101450),
 ('で', 84555),
 ('た', 83901),
 (' ', 79600),
 ('と', 76651),
 ('\n', 75755),
 ('「', 63155),
 ('」', 62506),
 ('し', 62485),
 ('\n\n', 51574),
 ('も', 47548),
 ('・', 41657),
 ('な', 36143),
 ('/', 35330),
 ('-', 30810),
 ('）', 29695),
 ('（', 29489),
 ('.', 27963),
 ('する', 27199),
 ('いる', 26871),
 ('ない', 25973),
 ('\u3000', 24848),
 ('こと', 24599),
 ('ます', 23614),
 ('さ', 22718),
 ('だ', 21838),
 ('から', 21775),
 ('です', 21643),
 ('！', 19974),
 ('れ', 19894),
 ('い', 17936),
 ('か', 17580),
 (':', 16201),
 ('や', 16179),
 ('日', 13651),
 ('：', 13616),
 ('という', 13517),
 ('人', 13277),
 ('ある', 13259),
 ('ん', 11700),
 ('『', 11483),
 ('など', 11473),
 ('？', 11385),
 ('』', 11345),
 ('う', 11279),
 ('00', 11246),
 ('】', 11162),
 ('【', 11158),
 ('1', 10966),
 ('なっ', 10776),
 ('よう', 10080),
 ('”', 9952),
 ('3', 9804),
 ('2', 9799),
 ('“', 9798),
 ('■', 9697),
 ('月', 9498),
 ('年', 9012),
 

In [147]:
# Text cleaning

doc_clean = []
for doc in documents:
    # Urls
    BAD_SYMBOL = re.compile(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Dates
    BAD_SYMBOL = re.compile(r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{4}")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Special characters
    BAD_SYMBOL = re.compile(r"[\n\*`\s:：「」、,。\.・/／\-ー\(\)（）!！『』？\?【】”“■\+＋〜~※&＆×％|●◯＜＞＝=★→]+")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Numbers
    BAD_SYMBOL = re.compile(r"\d+")
    doc = re.sub(BAD_SYMBOL, "NUMBER", doc)
    
    doc_clean.append(doc)

In [148]:
doc_clean[0]

'NY名物イベントが日本でも名店グルメを気軽に楽しむニュヨクでNUMBER年続いている食の祭典レストランウィクその日本版がダイナズクラブ特別協賛のもとNUMBER月NUMBER日よりスタトNUMBER月NUMBER日までの期間中青山六本木丸の内銀座横浜のエリアからラグジュアリレストランNUMBER店舗がこのイベントのために特別用意したランチメニュNUMBER円税サ別ディナNUMBER円税サ別を気軽に楽しめるとっておきのイベントです実行委員長には学校法人服部学園服部栄養専門学校理事長校長であり医学博士でもある服部幸應氏を迎え実行委員に石田純一さんLABETTOLAオナシェフ落合務氏フドアナリスト協会会長高賀右近氏つきぢ田村三代目田村隆氏にそして放送作家脚本家の小山薫堂さんなど食のスペシャリストたちが勢揃い参加レストランにはミシュランのフランス版東京版ともに星を獲得している吉野建シェフのレストランタテルヨシノ汐留や日本料理の名門つきぢ田村金田中庵赤坂璃宮にmikuniMARUNOUCHIなど日本を代表するレストランがずらりイベント期間のNUMBER月NUMBER日までは特別協賛のダイナズクラブカド会員またはシティバンクに口座を持つシティゴルドメンバが楽しめる先行期間となりますがその後は誰でも参加できるので日程のチェックは必須予約方法は必ず事前に各店舗に問合せを行いジャパンレストランウィクNUMBERでの予約であることを伝えればOK憧れていたレストランの料理をリズナブルにいただけるチャンスです極上の味とラグジュアリな空間を満喫そんな幸せを実感できるジャパンレストランウィクNUMBERにぜひ参加しててみてはいかがですかJAPANRESTAURANTWEEKNUMBER公式サイト'